I will be scraping per game stats and advanced stats from www.basketball-reference.com from seasons 1979/80 until the current one.
Then the MVP winners and DPOY winners from Wikipedia.

In [1]:
# importing the necessary libraries
import pandas as pd
from tqdm import tqdm
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# URL of stats per game
url = "https://www.basketball-reference.com/leagues/NBA_2022_per_game.html"

# now we create soup object
html = urlopen(url)
soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll('tr', limit=2)

# use getText() to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]

In [3]:
# we will scrape stats since 1979/80 season
years = [*range(1980,2023)]

# create empty df first
df_per_game = pd.DataFrame(columns=headers)
df_per_game['Year'] = None

# now we iterate over the years, get the stats and concatenate to the df
for year in tqdm(years):
    # URL page we will scraping
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

    html = urlopen(url)
    soup = BeautifulSoup(html)

    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    
    stats = pd.DataFrame(player_stats, columns=headers)
    stats['Year'] = year
    df_per_game = pd.concat([df_per_game,stats])

100%|██████████| 43/43 [01:00<00:00,  1.40s/it]


In [4]:
# do the same for advanced stats
url_advanced = "https://www.basketball-reference.com/leagues/NBA_2022_advanced.html"

html = urlopen(url_advanced)
soup = BeautifulSoup(html)

soup.findAll('tr', limit=2)

headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]

In [5]:
df_advanced = pd.DataFrame(columns=headers)
df_advanced['Year'] = None

for year in tqdm(years):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)

    html = urlopen(url)
    soup = BeautifulSoup(html)

    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    
    stats = pd.DataFrame(player_stats, columns=headers)
    stats['Year'] = year
    df_advanced = pd.concat([df_advanced,stats])

df_advanced.columns = ['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'empty1',
       'OWS', 'DWS', 'WS', 'WS/48', 'empty2', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Year']

# drop unnecessary columns, which we have in the per-game df
df_advanced.drop(columns=['Player','Pos','Age','Tm','G','MP','empty1','empty2','Year'], inplace=True)

100%|██████████| 43/43 [01:01<00:00,  1.43s/it]


In [6]:
# now we join the per-game and advanced stats together
df = pd.concat([df_per_game,df_advanced], axis=1)

In [7]:
# little cleaning of the player name column
df.dropna(subset=['Player'], inplace=True)
df['Player'] = df['Player'].str.split('*').str[0]

In [8]:
# let's see what it looks like
df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Kareem Abdul-Jabbar,C,32,LAL,82,,38.3,10.2,16.9,.604,...,15.7,24.1,9.5,5.3,14.8,.227,4.8,2.4,7.2,7.3
1,Tom Abernethy,PF,25,GSW,67,,18.2,2.3,4.7,.481,...,9.9,13.3,1.2,0.8,2.0,.080,-1.0,-0.2,-1.2,0.2
2,Alvan Adams,C,25,PHO,75,,28.9,6.2,11.7,.531,...,18.2,21.9,3.1,3.9,7.0,.155,1.7,1.9,3.6,3.1
3,Tiny Archibald,PG,31,BOS,80,80,35.8,4.8,9.9,.482,...,19.7,17.0,5.9,2.9,8.9,.148,1.4,-0.3,1.1,2.3
4,Dennis Awtrey,C,31,CHI,26,,21.5,1.0,2.3,.450,...,24.8,7.9,0.1,0.5,0.6,.053,-2.3,0.9,-1.4,0.1


In [9]:
# now let's fet the mvp winners from wikipedia
mvp_winners = pd.read_html('https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award#Winners')[5]

# again some data cleaning
mvp_winners['Season'] = mvp_winners['Season'].str.split('–').str[0]
mvp_winners['Season'] = mvp_winners['Season'].astype(int) + 1
mvp_winners['Player'] = mvp_winners['Player'].str.split('*').str[0]
mvp_winners['Player'] = mvp_winners['Player'].str.split('^').str[0]

# we get the winners from season 1979/80 onwards
mvp_winners = mvp_winners[mvp_winners['Season'] > 1979].copy()

# assign 1 which we then merge to the stats df
mvp_winners['MVP'] = 1
mvp_winners.head()

,Season,Player,Position,Nationality,Team,MVP
24,1980,Kareem Abdul-Jabbar,Center,United States,Los Angeles Lakers (3),1
25,1981,Julius Erving,Small forward,United States,Philadelphia 76ers (4),1
26,1982,Moses Malone,Center,United States,Houston Rockets (2),1
27,1983,Moses Malone,Center,United States,Philadelphia 76ers (5),1
28,1984,Larry Bird,Small forward,United States,Boston Celtics (8),1


In [10]:
# now we merge the MVP winners df
df = df.merge(mvp_winners[['Player','Season','MVP']], left_on=['Player','Year'], right_on=['Player','Season'], how='left')

# data cleaning
df.drop(columns='Season', inplace=True)
df['MVP'].fillna(0, inplace=True)
df['MVP'] = df['MVP'].astype(int)

In [11]:
# same for dpoy winners
dpoy_winners = pd.read_html('https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award')[6]
dpoy_winners['Season'] = dpoy_winners['Season'].str.split('–').str[0]
dpoy_winners['Season'] = dpoy_winners['Season'].astype(int) + 1
dpoy_winners['Player'] = dpoy_winners['Player'].str.split('*').str[0]
dpoy_winners['Player'] = dpoy_winners['Player'].str.split('^').str[0]
dpoy_winners['Player'] = dpoy_winners['Player'].str.split('[').str[0]
dpoy_winners = dpoy_winners[dpoy_winners['Season'] > 1979].copy()
dpoy_winners['DPOY'] = 1
dpoy_winners.head()

,Season,Player,Position,Nationality,Team,DPOY
0,1983,Sidney Moncrief,Shooting guard,United States,Milwaukee Bucks,1
1,1984,Sidney Moncrief,Shooting guard,United States,Milwaukee Bucks (2),1
2,1985,Mark Eaton,Center,United States,Utah Jazz,1
3,1986,Alvin Robertson,Shooting guard,United States,San Antonio Spurs,1
4,1987,Michael Cooper,Shooting guard,United States,Los Angeles Lakers,1


In [12]:
# merge the dpoy winners
df = df.merge(dpoy_winners[['Player','Season','DPOY']], left_on=['Player','Year'], right_on=['Player','Season'], how='left')
df.drop(columns='Season', inplace=True)

df['DPOY'].fillna(0, inplace=True)
df['DPOY'] = df['DPOY'].astype(int)

In [13]:
# change the numeric columns to numeric dtypes
numeric_columns = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'MVP', 'DPOY']

for column in numeric_columns:
    df[column] = pd.to_numeric(df[column])

In [14]:
# write to csv
df.to_csv('data/stats.csv', index=False)